In [2]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
from lib.models.vibe import VIBE_Demo, VIBE_Demo
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from lib.utils.demo_utils import download_ckpt

In [3]:
class VideoSilhouetteDataset(Dataset):
    def __init__(self, transform=None, target_transform=None):
        PATH = '/home/epinyoan/dataset/casia-b/dataset_b/all/silhouette_paths_320.npy'
        self.silhouette_paths = np.load(PATH)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.silhouette_paths)

    def __getitem__(self, idx):
        s_image = read_image(self.silhouette_paths[idx])
        file = self.silhouette_paths[idx].split('/')[-1].split('.')[0].split('-')
        raw_path = '/home/epinyoan/dataset/casia-b/dataset_b/all/images/'+'-'.join(file[:4])+'/'+file[4].zfill(6)+'.png'
        raw_image = read_image(raw_path).float()
        return raw_image, s_image

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = VIBE_Demo(
    seqlen=16,
    n_layers=2,
    hidden_size=1024,
    add_linear=True,
    use_residual=True,
    pretrained='../data/vibe_data/spin_model_checkpoint.pth.tar'
).to(device)

pretrained_file = '../data/vibe_data/vibe_model_w_3dpw.pth.tar'
ckpt = torch.load(pretrained_file)
print(f'Performance of pretrained model on 3DPW: {ckpt["performance"]}')
ckpt = ckpt['gen_state_dict']
model.load_state_dict(ckpt, strict=False)
model.eval()
''

=> loaded pretrained model from '../data/vibe_data/spin_model_checkpoint.pth.tar'
Performance of pretrained model on 3DPW: 52.00081691145897


''

In [5]:
vs_dataset = VideoSilhouetteDataset()
train_loader = DataLoader(vs_dataset, batch_size=64,
                                  num_workers=1, pin_memory=True, shuffle=True)

In [12]:
raw_image, s_image = next(iter(train_loader))
raw_image, s_image = raw_image.to(device), s_image.to(device)
raw_image = raw_image[:,:,:224,:224]

# hmr
print(raw_image.shape)
# feature = model.hmr.feature_extractor(raw_image)
output = model.hmr(raw_image)
print(output[0].keys())

# vibe
# raw_image = torch.unsqueeze(raw_image, 1)
# print(raw_image.shape)
# feature = model(raw_image)
# print(raw_image.shape)


torch.Size([64, 3, 224, 224])
dict_keys(['theta', 'verts', 'kp_2d', 'kp_3d'])
